In [ ]:
import requests
url = "https://www.seek.com.au/Data-Analyst-jobs/in-All-Perth-WA"


In [ ]:
response = requests.get(url)

In [ ]:
#save the html file
with open('seek.html', 'wb') as file:
    file.write(response.content)

In [ ]:
from bs4 import BeautifulSoup


In [ ]:
page_content = BeautifulSoup(response.content, 'html.parser')
print(page_content.prettify())

In [ ]:
# find all elements with attribute data-automation="jobTitle"
job_elems = page_content.find_all('a', attrs={'data-automation': 'jobTitle'})

for job_elem in job_elems:
    print(job_elem.text)
    print(job_elem['href'])
    print()

In [ ]:
new_url = "https://www.seek.com.au" + "/job/72886676?type=standard&ref=search-standalone"
response = requests.get(new_url)
page_content = BeautifulSoup(response.content, 'html.parser')
# print(page_content.prettify())

# find the element with attribute data-automation="jobAdDetails"
job_desc = page_content.find('div', attrs={'data-automation': 'jobAdDetails'})
print(job_desc.text)

In [ ]:
import requests

api_url = "https://www.seek.com.au/api/chalice-search/v4/search?siteKey=AU-Main&sourcesystem=houston&userqueryid=6d173ec636dcb32619e716be55fa2b43-8888113&userid=69a5310d-e99c-44e3-b5d1-dd794b0ef720&usersessionid=69a5310d-e99c-44e3-b5d1-dd794b0ef720&eventCaptureSessionId=69a5310d-e99c-44e3-b5d1-dd794b0ef720&where=Australia&page=1&seekSelectAllPages=true&keywords=Data+Analyst&include=seodata&locale=en-AU&seekerId=40762948&solId=5a6f5295-bb5f-4357-97c8-a69b1daadc54"

response = requests.get(api_url)
print(response.json())

In [ ]:
data = response.json()
data.keys()

In [ ]:
url = "https://www.seek.com.au/api/chalice-search/v4/related-search?zone=anz-1&keywords=data+analyst&siteKey=au&cluster=default&where=Australia"
response = requests.get(url)
base_url_api = "https://www.seek.com.au/api/chalice-search/v4/related-search?zone=anz-1&keywords="

data = response.json()

columns_mapping = {
    'keywords': 'Keywords',
    'totalJobs': 'Total Jobs'
}

# convert the data to a pandas dataframe
import pandas as pd
df = pd.json_normalize(data['relatedSearches'])

# only keep the columns we are interested in
df = df.rename(columns=columns_mapping)
df = df[columns_mapping.values()]

df.head()


In [ ]:
url = f"https://www.seek.com.au/api/chalice-search/v4/search?siteKey=AU-Main&sourcesystem=houston&where=Australia&page=2&seekSelectAllPages=true&keywords=Data+Analyst&hadPremiumListings=true&locale=en-AU"
response = requests.get(url)
print(response.json())

In [ ]:
import uuid

job_id = "74480328"
job_details_viewed_correlation_id = str(uuid.uuid4())
session_id = str(uuid.uuid4())
url = "https://www.seek.com.au/graphql"
payload = {
    "operationName": "GetJobDetails",
    "variables": {
        "jobId": job_id,
        "jobDetailsViewedCorrelationId": job_details_viewed_correlation_id,
        "sessionId": session_id,
        "zone": "anz-1",
        "locale": "en-AU",
        "languageCode": "en",
        "countryCode": "AU",
        "timezone": "Australia/Perth"
    },
    "query": "query GetJobDetails($jobId: ID!, $jobDetailsViewedCorrelationId: String!, $sessionId: String!, $zone: Zone!, $locale: Locale!, $languageCode: LanguageCodeIso!, $countryCode: CountryCodeIso2!, $timezone: Timezone!) {\n  jobDetails(\n    id: $jobId\n    tracking: {channel: \"WEB\", jobDetailsViewedCorrelationId: $jobDetailsViewedCorrelationId, sessionId: $sessionId}\n  ) {\n    job {\n      tracking {\n        adProductType\n        classificationInfo {\n          classificationId\n          classification\n          subClassificationId\n          subClassification\n          __typename\n        }\n        hasRoleRequirements\n        isPrivateAdvertiser\n        locationInfo {\n          area\n          location\n          locationIds\n          __typename\n        }\n        workTypeIds\n        postedTime\n        __typename\n      }\n      id\n      title\n      phoneNumber\n      isExpired\n      isLinkOut\n      contactMatches {\n        type\n        value\n        __typename\n      }\n      isVerified\n      abstract\n      content(platform: WEB)\n      status\n      listedAt {\n        label(context: JOB_POSTED, length: SHORT, timezone: $timezone, locale: $locale)\n        __typename\n      }\n      salary {\n        currencyLabel(zone: $zone)\n        label\n        __typename\n      }\n      shareLink(platform: WEB, zone: $zone, locale: $locale)\n      workTypes {\n        label(locale: $locale)\n        __typename\n      }\n      advertiser {\n        id\n        name(locale: $locale)\n        __typename\n      }\n      location {\n        label(locale: $locale, type: LONG)\n        __typename\n      }\n      classifications {\n        label(languageCode: $languageCode)\n        __typename\n      }\n      products {\n        branding {\n          id\n          cover {\n            url\n            __typename\n          }\n          thumbnailCover: cover(isThumbnail: true) {\n            url\n            __typename\n          }\n          logo {\n            url\n            __typename\n          }\n          __typename\n        }\n        bullets\n        questionnaire {\n          questions\n          __typename\n        }\n        video {\n          url\n          position\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    companyReviews(zone: $zone) {\n      id\n      name\n      fullName\n      rating\n      reviewCount\n      reviewsUrl\n      __typename\n    }\n    companySearchUrl(zone: $zone, languageCode: $languageCode)\n    learningInsights(platform: WEB, zone: $zone, locale: $locale) {\n      analytics\n      content\n      __typename\n    }\n    companyTags {\n      key(languageCode: $languageCode)\n      value\n      __typename\n    }\n    restrictedApplication(countryCode: $countryCode) {\n      label(locale: $locale)\n      __typename\n    }\n    sourcr {\n      image\n      imageMobile\n      link\n      __typename\n    }\n    __typename\n  }\n}\n"
}

response = requests.post(url, json=payload)

print(response.json())


In [ ]:
import pandas as pd
import json

#seek api base url
base_url = "https://www.seek.com.au/api/chalice-search/v4/search"
search_params = {
    "siteKey": "AU-Main",
    "sourcesystem": "houston",
    "where": "Australia",
    "page": 2,
    "seekSelectAllPages": True,
    "keywords": "Data Analyst",
    "locale": "en-AU"
}

response = requests.get(base_url, params=search_params)
data = response.json()

# save the data to a json file
with open('seek_data.json', 'w') as file:
    json.dump(data, file, indent=4)


jobs_data = data['data']

# get the json path of each attribute
columns_json_path_mapping = {
    "company": "advertiser.description",
    "classification": "classification.description",
    "subClassification": "subClassification.description",
    "listingDate": "listingDate",
    "location": "location",
    "country": "jobLocation.countryCode",
    "roleId": "roleId",
    "title": "title",
    "jobid": "id",
    "salary": "salary",
    "short_description": "teaser",
    "workType": "workType",
    "suburb": "suburb"
}

# pandas dataframe from json
df = pd.json_normalize(jobs_data)

# keep only the columns we need
df = df[columns_json_path_mapping.values()]

# rename the columns
df.columns = columns_json_path_mapping.keys()

df.head()

In [ ]:
metadata = data['solMetadata']
search_params = data['searchParams']



columns_json_path_mapping = {
    "pageNumber": "pageNumber",
    "totalJobCount": "totalJobCount"
}

# pandas dataframe from json
metadata_df = pd.json_normalize(metadata)

# keep only the columns we need
metadata_df = metadata_df[columns_json_path_mapping.values()]

# rename the columns
metadata_df.columns = columns_json_path_mapping.keys()

metadata_df.head()

In [ ]:
import json
import pandas as pd

file = "v5/response.json"

with open(file, 'r') as f:
    data = json.load(f)

jobs_data = data['data']

df = pd.json_normalize(jobs_data)

df.columns

In [ ]:
file = "v5/all_trends.csv"
df = pd.read_csv(file)
# convert the date column to datetime
df['searchDate'] = pd.to_datetime(df['searchDate']).dt.date

df.head()

In [ ]:
import os
import sqlalchemy
import yaml
import pandas as pd

config = yaml.safe_load(open('online/config.yml'))
# create a new db engine
server = config['db_credentials']['server']
database = config['db_credentials']['database']
user = config['db_credentials']['user']
password = os.getenv('DP101_PASSWORD')

sql_engine = sqlalchemy.create_engine(f"mssql+pyodbc://{user}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server",
                                  connect_args={'connect_timeout': 30})

# save the dataframe to the database
table_name = "jobs_trends"
df.to_sql(table_name, sql_engine, if_exists='replace', index=False)

In [ ]:
file = "v5/all_jobs.csv"

df = pd.read_csv(file)

df.head()

In [ ]:
df.dtypes

In [ ]:
# convert searchDate to datetime
df['searchDate'] = pd.to_datetime(df['searchDate']).dt.date

# convert jobPostedTime to datetime
df['jobPostedTime'] = pd.to_datetime(df['jobPostedTime']).dt.date

df.dtypes

In [15]:
import os
import sqlalchemy
import yaml
import pandas as pd


config = yaml.safe_load(open('online/config.yml'))
# create a new db engine
server = config['db_credentials']['server']
database = config['db_credentials']['database']
user = config['db_credentials']['user']
password = os.getenv('DP101_PASSWORD')

sql_engine = sqlalchemy.create_engine(f"mssql+pyodbc://{user}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server",
                                  connect_args={'connect_timeout': 30})


In [22]:
# catchup with job description

sql = """select jobid from jobs where jobid not in (select jobid from jobs_descriptions);"""

job_ids = pd.read_sql(sql, sql_engine)
print(job_ids.shape)
job_ids.head()

(398, 1)


,jobid
0,73862876
1,73935732
2,74179407
3,74316127
4,74316131


In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

job_ids_to_scrape = job_ids['jobid'].tolist()
all_job_desc = pd.DataFrame()
for job_id in job_ids_to_scrape:
    print(f"Getting job {job_id} description")
    seek_url = f"https://www.seek.com.au/job/{job_id}?ref=search-standalone&type=standout"
    response = requests.get(seek_url)
    page_content = BeautifulSoup(response.content, 'html.parser')
    job_desc = page_content.find('div', attrs={'data-automation': 'jobAdDetails'})
    if job_desc:
        job_desc = job_desc.text
        job_desc_df = pd.DataFrame({'jobid': [job_id], 'jobDescription': [job_desc]})
        all_job_desc = pd.concat([all_job_desc, job_desc_df], ignore_index=True)
    else:
        print(f"Job {job_id} not found")
        pass
all_job_desc.head()
# save the job descriptions to the database
table_name = "jobs_descriptions"
all_job_desc.to_sql(table_name, sql_engine, if_exists='append', index=False)

Getting job 73862876 description
Job 73862876 not found
Getting job 73935732 description
Job 73935732 not found
Getting job 74179407 description
Job 74179407 not found
Getting job 74316127 description
Getting job 74316131 description
Getting job 74316325 description
Getting job 74316365 description
Getting job 74316555 description
Getting job 74317726 description
Getting job 74317963 description
Getting job 74318035 description
Getting job 74318043 description
Getting job 74318117 description
Getting job 74318423 description
Getting job 74318507 description
Getting job 74319654 description
Getting job 74319953 description
Getting job 74320224 description
Getting job 74320360 description
Getting job 74320367 description
Getting job 74321616 description
Getting job 74322751 description
Getting job 74323081 description
Getting job 74323082 description
Getting job 74323491 description
Getting job 74323791 description
Getting job 74324912 description
Getting job 74324913 description
Getting

393

In [11]:
import requests

linkeding_base_url = r"https://www.linkedin.com/jobs/search/?keywords=data%20scientist&location=Australia&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true"

response = requests.get(linkeding_base_url)

with open('linkedin.html', 'wb') as file:
    file.write(response.content)


In [19]:
final_url = r"https://prosple-gw.global.ssl.fastly.net/internal?operationName=OpportunitiesSearch&variables=%7B%22parameters%22%3A%7B%22gid%22%3A%226%22%2C%22range%22%3A%7B%22offset%22%3A0%2C%22limit%22%3A20%7D%2C%22sortBy%22%3A%7B%22criteria%22%3A%22POPULARITY%22%2C%22direction%22%3A%22DESC%22%7D%2C%22subscriptionTierEpicVariation%22%3A%22THIRD_VARIATION%22%2C%22keywords%22%3A%22Internship%22%2C%22workRightLocation%22%3A%2229058%22%2C%22experiments%22%3A%7B%22searchByRole%22%3A%7B%22enabled%22%3Atrue%2C%22searchType%22%3A%22BEST_FIELDS%22%2C%22booleanQuery%22%3A%22MUST%22%2C%22weights%22%3A%7B%22title%22%3A150%2C%22description%22%3A50%2C%22opportunityType%22%3A30%2C%22employerName%22%3A100%7D%7D%2C%22bidKeyword%22%3A%7B%22enabled%22%3Afalse%7D%2C%22sortByBidPopularityKeyword%22%3A%22SECOND_VARIATION%22%7D%2C%22selectedStudyFieldFacets%22%3A%5B%7B%22id%22%3A%22502%22%2C%22childFacets%22%3A%5B%5D%7D%5D%2C%22selectedLocationFacets%22%3A%5B%7B%22id%22%3A%229692%22%2C%22childFacets%22%3A%5B%5D%7D%5D%7D%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22ee82239331107783c1f846a960fbf778ca4be2888cd9f965decd1ac4476c3a2f%22%7D%7D"

# decode the url and get the params
import urllib.parse
import json

params = urllib.parse.unquote(final_url)

# find the variables key
variables = params.split('variables=')[1].split('&extensions')[0]
extensions = params.split('extensions=')[1]
extensions = json.loads(extensions)
variables = json.loads(variables)

for key, value in variables['parameters'].items():
    print(key, value)

gid 6
range {'offset': 0, 'limit': 20}
sortBy {'criteria': 'POPULARITY', 'direction': 'DESC'}
subscriptionTierEpicVariation THIRD_VARIATION
keywords Internship
workRightLocation 29058
experiments {'searchByRole': {'enabled': True, 'searchType': 'BEST_FIELDS', 'booleanQuery': 'MUST', 'weights': {'title': 150, 'description': 50, 'opportunityType': 30, 'employerName': 100}}, 'bidKeyword': {'enabled': False}, 'sortByBidPopularityKeyword': 'SECOND_VARIATION'}
selectedStudyFieldFacets [{'id': '502', 'childFacets': []}]
selectedLocationFacets [{'id': '9692', 'childFacets': []}]


In [22]:
base_url = "https://prosple-gw.global.ssl.fastly.net/internal"
params = {
    "keywords": "Internship",
    "workRightLocation": "29058",
    "selectedStudyFieldFacets": [{"id": "502", "childFacets": []}],
    "selectedLocationFacets": [{"id": "9692", "childFacets": []}],
}
extensions = {
    "persistedQuery": {
        "version": 1,
        "sha256Hash": "ee82239331107783c1f846a960fbf778ca4be2888cd9f965decd1ac4476c3a2f"
    }
}
response = requests.post(base_url, json=params, params=extensions)

print(response)


<Response [400]>


In [1]:
import requests
import json
import pandas as pd


url = "https://au.gradconnection.com/api/campaignsearch/?query=IT&title=IT&disciplines=data-science-and-analytics&location=sydney%2CAU%2CCities&offset=0&limit=20&page=1"

response = requests.get(url)

data = response.json()

data

[{'campaigns': [{'id': '1f8c85e4-9d09-461a-9205-09550205f1d3',
    'title': 'Amazon Expression of Interest',
    'slug': 'amazon-amazon-expression-of-interest-18',
    'interval': {'start': '2024-09-22T02:12:00+00:00',
     'end': '2024-10-23T13:59:00+00:00'},
    'description': "Interested in joining one of our graduate or internship programs this year? We’re excited to be partnering with GradConnection to bring you Meet the Employers, a virtual speed networking event where we'll share all you need to know about getting a job with Amazon.",
    'disciplines': ['Accounting',
     'Actuarial Studies',
     'Administration',
     'Banking and Finance',
     'Business and Commerce',
     'Compliance',
     'Computer Science',
     'Consulting',
     'Cyber Security',
     'Data Science and Analytics',
     'Design and User Experience',
     'Economics',
     'Engineering - Software',
     'Funds Management',
     'Information Systems',
     'Information Technology',
     'Insurance',
    

In [5]:
for each_data in data:
    print(each_data['campaigns'])
    break

[{'id': '1f8c85e4-9d09-461a-9205-09550205f1d3', 'title': 'Amazon Expression of Interest', 'slug': 'amazon-amazon-expression-of-interest-18', 'interval': {'start': '2024-09-22T02:12:00+00:00', 'end': '2024-10-23T13:59:00+00:00'}, 'description': "Interested in joining one of our graduate or internship programs this year? We’re excited to be partnering with GradConnection to bring you Meet the Employers, a virtual speed networking event where we'll share all you need to know about getting a job with Amazon.", 'disciplines': ['Accounting', 'Actuarial Studies', 'Administration', 'Banking and Finance', 'Business and Commerce', 'Compliance', 'Computer Science', 'Consulting', 'Cyber Security', 'Data Science and Analytics', 'Design and User Experience', 'Economics', 'Engineering - Software', 'Funds Management', 'Information Systems', 'Information Technology', 'Insurance', 'Investment Banking', 'Management', 'Project Management', 'Telecommunications'], 'is_event': False, 'target_mode': 'url', 'r

In [7]:
data[0]['campaigns']

new_df = pd.json_normalize(data[0]['campaigns'])

new_df.head()

,id,title,slug,description,disciplines,is_event,target_mode,remote_option,apply_button_text,locations,...,display_disciplines,demographic,target_url,origin_target_url,item_type,can_summarise,additional_fields,salary,interval.start,interval.end
0,1f8c85e4-9d09-461a-9205-09550205f1d3,Amazon Expression of Interest,amazon-amazon-expression-of-interest-18,Interested in joining one of our graduate or i...,"[Accounting, Actuarial Studies, Administration...",False,url,remote_friendly,None,"[Canberra, Regional ACT, Regional New South Wa...",...,"[Accounting, Actuarial Studies, Administration...",None,/track-link/d7dcb5c0-00d3-4cb4-89f0-cc3b3f265498/,https://amazonspanz.splashthat.com/,keyword_searched_campaign,True,[],None,2024-09-22T02:12:00+00:00,2024-10-23T13:59:00+00:00


In [8]:
new_df.T

,0
id,1f8c85e4-9d09-461a-9205-09550205f1d3
title,Amazon Expression of Interest
slug,amazon-amazon-expression-of-interest-18
description,Interested in joining one of our graduate or i...
disciplines,"[Accounting, Actuarial Studies, Administration..."
is_event,False
target_mode,url
remote_option,remote_friendly
apply_button_text,None
locations,"[Canberra, Regional ACT, Regional New South Wa..."
